In [59]:
import numpy as np 
from implementations import *
from helpers import *
from helper_functions import *
from optimization import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [60]:
# load the training dataset
print("loading the training dataset")
DATA_TRAIN_PATH = 'train.csv'
y_tr, tx_tr, ids_tr = load_csv_data(DATA_TRAIN_PATH)
print("training dataset loaded")

loading the training dataset
training dataset loaded


# Data cleaning

In [61]:
txOpti, yOpti, _ = dataClean(tx_tr, y_tr)

# Train using ridge regression

In [62]:
losses = []
ws = []
txPoly = [] 

for i in range(4): 
    yOpti[i][np.where(yOpti[i] == -1)] = 0

    lambda_, degree = best_lambda_degree(yOpti[i], txOpti[i], 4, np.logspace(-6, 0, 30), np.arange(1, 5), 1)
    print("best lambda is ",lambda_," and best degree is ", degree)
    t = build_poly(txOpti[i], degree)
    w, loss = ridge_regression(yOpti[i], t, lambda_)
    losses.append(loss) 
    ws.append(w) 
    txPoly.append(t)
    print(f'end of the ridge_regression with loss={loss}')

best lambda is  1e-06  and best degree is  1
end of the ridge_regression with loss=0.06940528855719576
best lambda is  1e-06  and best degree is  1
end of the ridge_regression with loss=0.0908194056759205
best lambda is  1e-06  and best degree is  1
end of the ridge_regression with loss=0.08717399062025641
best lambda is  1e-06  and best degree is  1
end of the ridge_regression with loss=0.08878703991906979


In [70]:
print(w.shape)

(31,)


# Test accuracy

In [71]:
labels = []
accs = []
for i in range(4): 
    label = predict(ws[i], txPoly[i])
    yOpti[i][np.where(yOpti[i] == 0)] = -1
    acc = calculate_accuracy(yOpti[i], label)
    print("the accuracy on the train set is ", acc)
    accs.append(acc)
    labels.append(label)

accTot = (accs[0] + accs[1] + accs[2] + accs[3])/4
print("the total accuracy on the train set is ", accTot)

the accuracy on the train set is  0.8131574469788716
the accuracy on the train set is  0.7320875889817394
the accuracy on the train set is  0.747513845054487
the accuracy on the train set is  0.7285237321783071
the total accuracy on the train set is  0.7553206532983512


# Predict test set

In [72]:
# load the test dataset
print("loading the test dataset")
DATA_TEST_PATH = 'test.csv'
y_te, tx_te, ids_te = load_csv_data(DATA_TEST_PATH)
print("test dataset loaded")

loading the test dataset
test dataset loaded


In [73]:
txOpti_te, yOpti_te, idsOpti_te = dataClean(tx_te, y_te)

In [77]:
y_pred = np.zeros(len(ids_te))
degree = 1

for i in range(4):
    t = build_poly(txOpti_te[i], degree)
    predicted = predict_logistic(ws[i], t)
    predicted[np.where(predicted == 0)] = -1
    y_pred[idsOpti_te[i]] = predicted

# Generated submission file

In [78]:
OUTPUT_PATH = 'ridge-submission'
create_csv_submission(ids_te, y_pred, OUTPUT_PATH)

0.8329537975285004
